# Content Safety on container

In [1]:
#%pip install azure-ai-contentsafety

In [2]:
import docker
import os
import requests

from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory
from dotenv import load_dotenv

C:\Users\seretkow\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
load_dotenv("azure.env")

key = os.getenv("key")

In [4]:
docker_model_endpoint = "http://localhost:5000"

In [5]:
docker_client = docker.from_env()
docker_client

In [6]:
print("Active Docker container(s):\n")

for idx2, container in enumerate(docker_client.containers.list(), start=1):
    print(f"{idx2} - Docker container name: {container.name}")
    print(f"Container image: {container.image}")
    print(f"Container id: {container.id}\n")

Active Docker container(s):

1 - Docker container name: peaceful_heisenberg
Container image: <Image: 'mcr.microsoft.com/azure-cognitive-services/contentsafety/text-analyze:latest'>
Container id: 7de950ca140439d163bd279a505cd2fc9fe90afd359ffe62bfe9f73b54a05519



In [7]:
def analyze_text(yourtext):
    """
    Analyzes the given text using the Content Safety Client.

    Parameters:
    yourtext (str): The text to be analyzed.

    Returns:
    dict: A dictionary containing the categories analysis of the text.

    Raises:
    HttpResponseError: If the text analysis fails, an error message and code 
    are printed, and the exception is raised.
    """
    # client
    client = ContentSafetyClient(docker_model_endpoint,
                                 AzureKeyCredential(key))
    # request
    request = AnalyzeTextOptions(text=yourtext)
    try:
        response = client.analyze_text(request)
    except HttpResponseError as e:
        print("Analyze text failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    return response["categoriesAnalysis"]

In [8]:
analyze_text("I will kill this thing")

[{'category': 'selfHarm', 'severity': 0},
 {'category': 'violence', 'severity': 3},
 {'category': 'hate', 'severity': 0},
 {'category': 'sexual', 'severity': 0}]

In [9]:
analyze_text("Hello")

[{'category': 'selfHarm', 'severity': 0},
 {'category': 'violence', 'severity': 0},
 {'category': 'hate', 'severity': 0},
 {'category': 'sexual', 'severity': 0}]